# PII Masking
## Membros:
- Jean Rodrigues Rocha - RA: 813581
- Leonardo Prado Silva - RA: 813169
- Rafael Gimenez Barbeta - RA: 804318
- Vanderlei Guilherme Andrade de Assis - 802162
- Wilker Silva Ribeiro - 813291


In [1]:
import re
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import ast
import re
from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline
from transformers.pipelines import AggregationStrategy
from datasets import Dataset, load_dataset
from evaluate import load
from seqeval.metrics import classification_report as seqeval_classification_report
from seqeval.metrics import f1_score as seqeval_f1, accuracy_score as seqeval_accuracy
from sklearn.metrics import classification_report as sklearn_classification_report
import numpy as np

In [2]:
from scripts.preprocessamento import *
from scripts.modelo_transformer import *
from scripts.modelo_regras import *

In [3]:
df = pd.read_json("dataset_original.jsonl", lines=True)
df = remover_tags_nao_utilizadas(df)
df_traduzido = traduzir_para_portugues(df, carregar_traduzido=True) #uma parte da função que foi usada para fazer a tradução foi comentada por erros de versão, e por demorar muito para executar.

amostra = selecionar_tuplas_validacao_manual(df_traduzido)
df_validacao = carregar_tuplas_validacao_manual()
resultados_bleu, resultados_chrf = avaliar_qualidade_traducao(amostra,df_validacao)
print(resultados_bleu)
print(resultados_chrf)

{'bleu': 0.5796358578058415, 'precisions': [0.7900994904149479, 0.6390902274431393, 0.5225457356351456, 0.42781175219356554], 'brevity_penalty': 1.0, 'length_ratio': 1.0215666831928607, 'translation_length': 4121, 'reference_length': 4034}
{'score': 76.92308065643913, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [4]:
df = pd.read_csv("dataset_traduzido.csv")

df['bert_tokens_pt'] = df['bert_tokens_pt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['bio_tags_pt'] = df['bio_tags_pt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [5]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [6]:
label_list = list(set(tag for sublist in df['bio_tags_pt'] for tag in sublist))
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

def align_token_based_labels(examples):
    tokenized_inputs = tokenizer(
        examples["source_text_pt"],
        truncation=True,
        max_length=512
    )
    
    all_labels = []
    for i, labels_for_sentence in enumerate(examples["bio_tags_pt"]):
        num_tokens = len(tokenized_inputs['input_ids'][i])

        num_labels = len(labels_for_sentence)

        if num_tokens != num_labels + 2:
            pass

        new_labels = [-100]
        new_labels.extend([label_to_id[label] for label in labels_for_sentence])
        new_labels.append(-100)
        all_labels.append(new_labels[:num_tokens])

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


hf_dataset = Dataset.from_pandas(df)
tokenized_datasets = hf_dataset.map(align_token_based_labels, batched=True)

train_test_split = tokenized_datasets.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

df_regex = pd.DataFrame(eval_dataset)
df_transformer = pd.DataFrame(eval_dataset)

Map:   0%|          | 0/15057 [00:00<?, ? examples/s]

In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report as sklearn_classification_report
from seqeval.metrics import classification_report as seqeval_classification_report, f1_score as seqeval_f1, accuracy_score as seqeval_accuracy

model = AutoModelForTokenClassification.from_pretrained(
    model_name, 
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[pred] for pred, lab in zip(pred_seq, label_seq) if lab != -100]for pred_seq, label_seq in zip(predictions, labels)]
    true_labels = [[label_list[lab] for pred, lab in zip(pred_seq, label_seq) if lab != -100]for pred_seq, label_seq in zip(predictions, labels)]

    flat_true = [t for seq in true_labels for t in seq]
    flat_pred = [t for seq in true_predictions for t in seq]

    entity_report = seqeval_classification_report(true_labels, true_predictions, output_dict=True, zero_division=0)
    entity_f1 = seqeval_f1(true_labels, true_predictions)
    entity_accuracy = seqeval_accuracy(true_labels, true_predictions)
    entity_precision = entity_report["macro avg"]["precision"]
    entity_recall = entity_report["macro avg"]["recall"]
    
    token_report = sklearn_classification_report(flat_true, flat_pred, output_dict=True, zero_division=0)
    token_f1 = token_report["weighted avg"]["f1-score"]
    token_accuracy = token_report["accuracy"]
    token_precision = token_report["weighted avg"]["precision"]
    token_recall = token_report["weighted avg"]["recall"]

    return {
        "entity_f1": entity_f1,
        "entity_accuracy": entity_accuracy,
        "entity_precision": entity_precision,
        "entity_recall": entity_recall,
        
        "token_f1": token_f1,
        "token_accuracy": token_accuracy,
        "token_precision": token_precision,
        "token_recall": token_recall,
    }

args = TrainingArguments(
    output_dir="PII-detection-train",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train(resume_from_checkpoint=False) 

trainer.save_model("PII-detection-final")
tokenizer.save_pretrained("PII-detection-final")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.569200
1000,0.135300
1500,0.099800
2000,0.085900
2500,0.078900
3000,0.078400
3500,0.073500
4000,0.065900
4500,0.067000
5000,0.075200


('PII-detection-final\\tokenizer_config.json',
 'PII-detection-final\\special_tokens_map.json',
 'PII-detection-final\\vocab.txt',
 'PII-detection-final\\added_tokens.json',
 'PII-detection-final\\tokenizer.json')

In [8]:
results_transformer = trainer.evaluate()
for key, value in results_transformer.items():
    print(f"{key}: {value:.4f}")

eval_loss: 0.1675
eval_entity_f1: 0.9343
eval_entity_accuracy: 0.9695
eval_entity_precision: 0.9428
eval_entity_recall: 0.9433
eval_token_f1: 0.9693
eval_token_accuracy: 0.9695
eval_token_precision: 0.9692
eval_token_recall: 0.9695
eval_runtime: 7.5382
eval_samples_per_second: 199.7820
eval_steps_per_second: 25.0720
epoch: 10.0000


In [7]:
model_path = "PII-detection-final"

ner_pipeline_transformers = pipeline("ner", model=model_path, tokenizer=model_path, aggregation_strategy=AggregationStrategy.SIMPLE)

df_transformer["processed_text"] = df_transformer["source_text_pt"].apply(ner_pipeline_transformers)
df_transformer["processed_text"] = df_transformer.apply(lambda row: process_text_transformer(row["source_text_pt"], row["processed_text"]),axis=1)

for i in range(len(df_transformer["target_text_pt"])):
    if df_transformer["target_text_pt"][i] != df_transformer["processed_text"][i]:
        print(f"ID: {df_transformer.index[i]}")
        print(f"Original : {df_transformer["target_text_pt"][i]}")
        print(f"Resultado: {df_transformer["processed_text"][i]}")
        print()

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ID: 7
Original : Para garantir que temos o indivíduo certo, por favor verifica seu agente de usuário atual [USERAGENT] associado com o endereço IP [IPV6].
Resultado: Para garantir que temos o indivíduo certo, por favor verifica seu agente de usuário atual [USERAGENT] associado com o endereço IP [IP].

ID: 18
Original : Querido [USUARIO], estamos tentando chegar a você sobre a sua definição para [DATANASCIMENTO]. A nossa clínica está localizada em [NUMEROPREDIO] [CEP]. Se você precisa se isolar devido à recente pandemia, entre em contato conosco no [IMEI].
Resultado: Querido [USUARIO], estamos tentando chegar a você sobre a sua definição para [DATANASCIMENTO]. A nossa clínica está localizada em [NUMEROTELEFONE]. Se você precisa se isolar devido à recente pandemia, entre em contato conosco no [IMEI].

ID: 21
Original : Caro [PRIMEIRONOME], entre as suas informações de faturamento para o curso de Pesquisa Clínica Avançada em nossa página segura no [IP].
Resultado: Caro [PRIMEIRONOME], ent

# Modelo baseado em Regras

In [8]:
ESTADOS_BRASILEIROS = [
    'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO',
    'Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará', 'Distrito Federal', 'Espírito Santo', 'Goiás', 'Maranhão', 'Mato Grosso', 'Mato Grosso do Sul', 'Minas Gerais', 'Pará',
    'Paraíba', 'Paraná', 'Pernambuco', 'Piauí', 'Rio de Janeiro', 'Rio Grande do Norte', 'Rio Grande do Sul', 'Rondônia', 'Roraima', 'Santa Catarina', 'São Paulo', 'Sergipe', 'Tocantins'
]

PRIMEIROS_NOMES = ['João', 'Maria', 'José', 'Ana', 'Carlos', 'Francisca', 'Paulo', 'Antônio','Luiz', 'Adriana', 'Pedro', 'Juliana', 'Marcos', 'Marcia', 'Raimundo']
ULTIMOS_NOMES = ['Silva', 'Santos', 'Oliveira', 'Souza', 'Rodrigues', 'Ferreira', 'Alves', 'Pereira', 'Lima', 'Azevedo', 'Araújo', 'Barbosa', 'Ribeiro']
CIDADES = ['São Paulo', 'Rio de Janeiro', 'Brasília', 'Salvador', 'Fortaleza', 'Belo Horizonte', 'Manaus', 'Curitiba', 'Recife', 'Goiânia']

matcher_patterns = [
    {"label": "ESTADO", "pattern": [
        {"LOWER": {"IN": ["estado", "state", "estado:", "state:"]}},
        {"LOWER": "de", "OP": "?"},
        {"TEXT": {"IN": ESTADOS_BRASILEIROS}}
    ]},

    {"label": "PRIMEIRONOME", "pattern": [
        {"LOWER": {"IN": ["nome", "name", "cliente", "caro", "nome:", "name:", "cliente:"]}},
        {"IS_PUNCT": True, "OP": "*"},
        {"TEXT": {"IN": PRIMEIROS_NOMES}}
    ]},

    {"label": "ULTIMONOME", "pattern": [
        {"LOWER": {"IN": ["nome", "name", "cliente", "caro", "nome:", "name:", "cliente:"]}},
        {"OP": "*", "IS_PUNCT": False},
        {"TEXT": {"IN": ULTIMOS_NOMES}}
    ]},
    
    {"label": "CIDADE", "pattern": [
        {"LOWER": {"IN": ["cidade", "cidade:"]}},
        {"LOWER": "de", "OP": "?"},
        {"TEXT": {"IN": CIDADES}}
    ]},

    {"label": "USUARIO", "pattern": [
        {"LOWER": {"IN": ["usuario", "user", "username", "usuário", "usuario:", "user:", "username:", "usuário:"]}},
        {"OP": "*", "IS_PUNCT": False, "LENGTH": {">": 0}},
        {"TEXT": {"REGEX": r"^[a-zA-Z][a-zA-Z0-9_\.]{2,19}$"}}
    ]},
    
    {"label": "SENHA", "pattern": [
        {"LOWER": {"IN": ["senha", "password", "pass", "pwd"]}},
        {"OP": "*", "IS_PUNCT": False},
        {"IS_PUNCT": False},
        {"TEXT": {"IN": list(carregar_senhas_rockyou())}}
    ]},

    {"label": "NUMEROCONTA", "pattern": [
        {"LOWER": {"IN": ["conta", "account", "numero da conta"]}},
        {"OP": "*", "IS_PUNCT": False, "IS_DIGIT": False},
        {"TEXT": {"REGEX": r"^\d{6,12}$"}}
    ]},

    {"label": "PIN", "pattern": [
        {"LOWER": {"IN": ["pin", "codigo", "código"]}},
        {"OP": "*", "IS_PUNCT": False, "IS_DIGIT": False},
        {"TEXT": {"REGEX": r"^\d{4,6}$"}}
    ]},

    {"label": "CVVCARTAOCREDITO", "pattern": [
        {"LOWER": {"IN": ["cvv", "código"]}},
        {"LOWER": "de", "OP": "?"},
        {"LOWER": "segurança", "OP": "?"},
        {"OP": "*", "IS_PUNCT": False, "IS_DIGIT": False},
        {"TEXT": {"REGEX": r"^\d{3,4}$"}}
    ]},

    {"label": "NUMEROPREDIO", "pattern": [
        {"LOWER": {"IN": ["endereço", "rua", "av.", "avenida"]}},
        {"OP": "+", "IS_PUNCT": False},
        {"IS_PUNCT": True, "TEXT": ","},
        {"TEXT": {"REGEX": r"^\d{1,5}$"}}
    ]}
]

ruler_patterns = [
    {'label': 'IPV4', 'pattern': [{'TEXT': {'REGEX': r"\b(?:(?:25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.){3}(?:25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\b"}}]},
    {'label': 'CPF', 'pattern': [{'TEXT': {'REGEX': r"\b\d{3}\.?\d{3}\.?\d{3}-?\d{2}\b"}}]},
    {'label': 'BIC', 'pattern': [{'TEXT': {'REGEX': r"\b[A-Z]{4}[A-Z]{2}[A-Z0-9]{2}([A-Z0-9]{3})?\b"}}]},
    {'label': 'IMEI', 'pattern': [{'TEXT': {'REGEX': r"\b\d{15}\b"}}]},
    {'label': 'EMAIL', 'pattern': [{'TEXT': {'REGEX': r"\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b"}}]},
    {'label': 'CEP', 'pattern': [{'TEXT': {'REGEX': r"\b\d{5}-\d{3}\b"}}]},
    {'label': 'IPV6', 'pattern': [{'TEXT': {'REGEX': r"\b(?:[0-9a-fA-F]{1,4}:){7}[0-9a-fA-F]{1,4}\b"}}]},
    {'label': 'MAC', 'pattern': [{'TEXT': {'REGEX': r"\b(?:[0-9A-Fa-f]{2}[:-]){5}[0-9A-Fa-f]{2}\b"}}]},
    {'label': 'VINVEICULO', 'pattern': [{'TEXT': {'REGEX': r"\b[A-HJ-NPR-Z0-9]{17}\b"}}]},
    {'label': 'ENDERECOLITECOIN', 'pattern': [{'TEXT': {'REGEX': r"\b[L3M][a-km-zA-HJ-NP-Z1-9]{26,33}\b"}}]},
    {'label': 'ENDERECOETHER', 'pattern': [{'TEXT': {'REGEX': r"\b0x[a-fA-F0-9]{40}\b"}}]},
    {'label': 'ENDERECOBITCOIN', 'pattern': [{'TEXT': {'REGEX': r"\b[13][a-km-zA-HJ-NP-Z1-9]{25,34}\b"}}]},
    {'label': 'ENDERECOSECUNDARIO', 'pattern': [{'TEXT': {'REGEX': r"(?i)\b(?:apt\.?|apto\.?|bloco|andar|unidade)\s*\d+\b"}}]},
    {'label': 'ENDERECOGPSPROXIMO', 'pattern': [{'TEXT': {'REGEX': r"\b\[\s*-?\d{1,3}\.\d+,\s*-?\d{1,3}\.\d+\s*\]"}}]},
    {'label': 'NUMEROCARTAOCREDITO', 'pattern': [{'TEXT': {'REGEX': r"\b(?:\d{4}[\s-]?){3}\d{4}\b"}}]},
    {'label': 'IBAN', 'pattern': [{'TEXT': {'REGEX': r"\b[A-Z]{2}\d{2}[A-Z0-9]{11,30}\b"}}]},
    {'label': 'VRMVEICULO', 'pattern': [{'TEXT': {'REGEX': r"\b[A-Z]{3}-?\d{4}\b"}}]},
    {'label': 'DATANASCIMENTO', 'pattern': [{'TEXT': {'REGEX': r"\b\d{1,2}/\d{1,2}/\d{4}\b"}}]},
    {'label': 'USERAGENT', 'pattern': [{'TEXT': {'REGEX': r"\bMozilla/\d+\.\d+\s*\([^)]+\)\s*[^\n\[\]]+"}}]},
    {'label': 'PREFIXO', 'pattern': [{'TEXT': {'REGEX': r"(?i)\b(?:sr\.?|sra\.?|dr\.?|dra\.?|srta\.?)\b"}}]},
    {'label': 'NUMEROTELEFONE', 'pattern': [{'TEXT': {'REGEX': r"\b\(?\d{2}\)?[\s-]?\d{4,5}[\s-]?\d{4}\b"}}]},
    {'label': 'RUA', 'pattern': [{'TEXT': {'REGEX': r"\b(?:Rua|Av\.?|Avenida)\s+[^,]+"}}]}
]

@Language.component("context_entity_matcher")
def context_entity_matcher_component(doc):
    matcher = Matcher(nlp.vocab)
    
    for rule in matcher_patterns:
        matcher.add(rule["label"], [rule["pattern"]])

    matches = matcher(doc)
    
    used_tokens = set()
    final_ents = []
    
    sorted_matches = sorted(matches, key=lambda m: m[2] - m[1], reverse=True)

    for match_id, start, end in sorted_matches:
        if any(token_index in used_tokens for token_index in range(start, end)):
            continue
            
        label_str = nlp.vocab.strings[match_id]
        
        entity_span = Span(doc, end - 1, end, label=label_str)
        final_ents.append(entity_span)
        
        used_tokens.update(range(start, end))

    existing_ents = list(doc.ents)
    existing_tokens = set()
    for ent in existing_ents:
        existing_tokens.update(range(ent.start, ent.end))

    for ent in final_ents:
        if not any(token.i in existing_tokens for token in ent):
            existing_ents.append(ent)
            
    doc.ents = existing_ents
    return doc

nlp = spacy.blank("pt")

ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(ruler_patterns)

nlp.add_pipe("context_entity_matcher", after="entity_ruler")

for rule in ruler_patterns:
    component_name = make_regex_component(rule['label'], rule['pattern'][0]['TEXT']['REGEX'])
    if component_name not in nlp.pipe_names:
        nlp.add_pipe(component_name, after="entity_ruler")

print("Pipeline components:", nlp.pipe_names)
print("-" * 30)


docs = nlp.pipe(df_regex['source_text_pt'])

all_spans = []
for n, doc in enumerate(docs):
    if n % 1000 == 0:
        print(f"{n} / {len(df_regex['source_text_pt'])}")
    spans_for_doc = []
    for ent in doc.ents:
        spans_for_doc.append([ent.start_char, ent.end_char, ent.label_])
    all_spans.append(spans_for_doc)

df_regex['spans_from_source_pt'] = all_spans

Pipeline components: ['entity_ruler', 'regex_rua_detector', 'regex_numerotelefone_detector', 'regex_prefixo_detector', 'regex_useragent_detector', 'regex_datanascimento_detector', 'regex_vrmveiculo_detector', 'regex_iban_detector', 'regex_numerocartaocredito_detector', 'regex_enderecogpsproximo_detector', 'regex_enderecosecundario_detector', 'regex_enderecobitcoin_detector', 'regex_enderecoether_detector', 'regex_enderecolitecoin_detector', 'regex_vinveiculo_detector', 'regex_mac_detector', 'regex_ipv6_detector', 'regex_cep_detector', 'regex_email_detector', 'regex_imei_detector', 'regex_bic_detector', 'regex_cpf_detector', 'regex_ipv4_detector', 'context_entity_matcher']
------------------------------
0 / 1506
1000 / 1506


In [9]:
df_regex['bio_tags_regex_pt'] = df_regex.apply(get_bio_tags_regex, axis=1, args=(tokenizer,))
df_regex['bio_tags_pt'] = df_regex['bio_tags_pt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_regex['bio_tags_regex_pt'] = df_regex['bio_tags_regex_pt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

results = calculate_metrics_regex(df_regex['bio_tags_pt'], df_regex['bio_tags_regex_pt'])

for key, value in results.items():
    print(f"{key}: {value:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


eval_entity_f1: 0.4583
eval_entity_accuracy: 0.8042
eval_entity_precision: 0.5305
eval_entity_recall: 0.3993
eval_token_f1: 0.7463
eval_token_accuracy: 0.8042
eval_token_precision: 0.7364
eval_token_recall: 0.8042


In [10]:
df_regex['spans_from_source_pt'] = df_regex['spans_from_source_pt'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_regex['processed_text'] = df_regex.apply(lambda row: process_text_regex(row['source_text_pt'], row['spans_from_source_pt']),axis=1)

for i in range(len(df_transformer["target_text_pt"])):
    if df_regex["target_text_pt"][i] != df_regex["processed_text"][i]:
        print(f"ID: {df_regex.index[i]}")
        print(f"Original : {df_regex["target_text_pt"][i]}")
        print(f"Resultado: {df_regex["processed_text"][i]}")
        print()

ID: 0
Original : Permanecer seguro online significa ter cuidado com a partilha de especificidades de suas credenciais, como [CEP], [IP] ou [IPV6].
Resultado: Permanecer seguro online significa ter cuidado com a partilha de especificidades de suas credenciais, como [CEP], [IPV6] ou [IPV6].

ID: 1
Original : Encontrou possíveis problemas de segurança com o IP [IP], rastreado para um arquivo do paciente. Felizmente, senhas como [SENHA] protegem informações sensíveis.
Resultado: Encontrou possíveis problemas de segurança com o IP [IPV4], rastreado para um arquivo do paciente. Felizmente, senhas como af7e3zlPoFGm protegem informações sensíveis.

ID: 2
Original : Olá [PRIMEIRONOME], estamos chegando a você sobre sua próxima sessão de tratamento no [DATANASCIMENTO]. Por favor, confirme sua disponibilidade.
Resultado: Olá Augusto, estamos chegando a você sobre sua próxima sessão de tratamento no 1989/05/10. Por favor, confirme sua disponibilidade.

ID: 4
Original : Os recursos-chave incluirão 